In [ ]:
%%capture --no-stderr
%pip install openai==1.3.7 python-dotenv

In [32]:
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key=api_key)

In [33]:
def query_openai(system_prompt, message):
    messages = []
    if system_prompt:
        messages.append({
            "role": "system",
            "content": system_prompt,
        })
    messages.append({
        "role": "user",
        "content": message,
    })
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0.2,
        messages=messages
    )
    output = completion.choices[0].message
    return output.content


def query_and_print(system_prompt, message):
    print("A:")
    print(query_openai(system_prompt, message))

## Self Attention

`best place` 보다 `tourist atrraction`이 질문의 의도에 더 명확하게 부합하는 embedding 값을 제공한다.

In [34]:
query_and_print("", "What are the best places in Korea?")

A:
Korea is rich in culture, history, and natural beauty, offering a wide range of attractions for visitors. Here are some of the best places to explore in South Korea:

### 1. **Seoul**
   - **Gyeongbokgung Palace**: The largest of the Five Grand Palaces, known for its beautiful architecture and changing of the guard ceremony.
   - **Bukchon Hanok Village**: A traditional Korean village with hanok (traditional houses) that offers a glimpse into Korea's past.
   - **Myeongdong**: A bustling shopping district famous for cosmetics, fashion, and street food.
   - **N Seoul Tower**: Offers panoramic views of the city, especially beautiful at night.

### 2. **Busan**
   - **Haeundae Beach**: One of the most popular beaches in Korea, perfect for sunbathing and water sports.
   - **Gamcheon Culture Village**: Known for its colorful houses and artistic vibe, it's a great place for photography.
   - **Jagalchi Fish Market**: The largest seafood market in Korea, where you can enjoy fresh seafood

In [35]:
query_and_print("", "What are the tourist attractions in Korea?")

A:
South Korea is rich in cultural heritage, natural beauty, and modern attractions. Here are some of the top tourist attractions in the country:

### Historical and Cultural Sites
1. **Gyeongbokgung Palace** (Seoul) - The largest of the Five Grand Palaces, known for its beautiful architecture and the changing of the guard ceremony.
2. **Changdeokgung Palace** (Seoul) - A UNESCO World Heritage site famous for its Secret Garden and harmonious design with nature.
3. **Bukchon Hanok Village** (Seoul) - A traditional village with well-preserved hanok (Korean traditional houses) offering a glimpse into Korea's past.
4. **Jongmyo Shrine** (Seoul) - A Confucian shrine dedicated to the deceased kings and queens of the Joseon Dynasty, also a UNESCO World Heritage site.
5. **Hwaseong Fortress** (Suwon) - A UNESCO World Heritage site featuring impressive walls and gates, built in the late 18th century.

### Natural Attractions
6. **Seoraksan National Park** - Known for its stunning mountain scene

## Few Shot

In [36]:
# 영어를 한국어로 번역하는 5-shot prompt 를 작성하고 "dog"를 번역해보세요.

query_and_print("", """
다음 영어 단어들을 한국어로 번역해주세요:

1. sun: 태양
2. house: 집
3. friend: 친구
4. flower: 꽃
5. computer: 컴퓨터

위의 예시를 참고하여 다음 단어를 한국어로 번역해주세요:

dog:              
""")

A:
dog: 개


In [37]:
# 영화 리뷰에 대한 sentiment (positive or negative) 를 결정하는 5-shot prompt 를 작성하고 "The storyline was dull and uninspiring." 에 대한 결과를 확인해보세요.

query_and_print("", """
다음 영화 리뷰들의 감정(sentiment)이 긍정적(positive)인지 부정적(negative)인지 판단해주세요:

1. "This movie was an absolute masterpiece with stunning visuals and a gripping plot.": positive
2. "I couldn't sit through the entire film. It was boring and poorly acted.": negative
3. "The special effects were impressive, but the story lacked depth and character development.": negative
4. "I was pleasantly surprised by this film. It exceeded my expectations in every way.": positive
5. "While not perfect, the movie had its moments and kept me entertained throughout.": positive

위의 예시를 참고하여 다음 영화 리뷰의 감정(sentiment)을 판단해주세요:

"The storyline was dull and uninspiring.":
""")

A:
negative


In [38]:
# 자연어를 SQL 쿼리로 바꿔주는 few-shot prompt 를 작성해보세요.

query_and_print("", """
다음 자연어 요청을 SQL 쿼리로 변환하세요:

1. 요청: "직원 테이블에서 급여가 50,000 달러 이상인 모든 직원을 보여주세요."
   SQL: SELECT * FROM employees WHERE salary >= 50000;

2. 요청: "제품 테이블에서 재고가 10개 미만인 모든 제품의 이름과 재고량을 나열하세요."
   SQL: SELECT product_name, stock FROM products WHERE stock < 10;

3. 요청: "주문 테이블에서 지난 7일 동안의 총 주문 금액을 계산하세요."
   SQL: SELECT SUM(total_amount) FROM orders WHERE order_date >= DATE_SUB(CURDATE(), INTERVAL 7 DAY);

4. 요청: "고객 테이블에서 각 도시별 고객 수를 구하고, 고객 수가 많은 순서로 정렬하세요."
   SQL: SELECT city, COUNT(*) as customer_count FROM customers GROUP BY city ORDER BY customer_count DESC;

5. 요청: "직원 테이블에서 부서별 평균 급여를 계산하고, 평균 급여가 60,000 달러 이상인 부서만 표시하세요."
   SQL: SELECT department, AVG(salary) as avg_salary FROM employees GROUP BY department HAVING avg_salary >= 60000;

위의 예시를 참고하여 다음 자연어 요청을 SQL 쿼리로 변환하세요:

요청: "판매 테이블에서 2023년의 월별 총 판매액을 계산하세요."
SQL:
""")

A:
SQL: SELECT MONTH(sale_date) as month, SUM(sale_amount) as total_sales FROM sales WHERE YEAR(sale_date) = 2023 GROUP BY MONTH(sale_date);


## Chain of Thought

다음 문제를 푸는 Chain of thought prompt 를 작성해보세요.

```
# Simple - 1
Solve the following problem step-by-step: 23 + 47

Step-by-step solution:
1. Write your Prompt
2. Write your Prompt
3. Write your Prompt
4. Write your Prompt

Answer: 70
```

```
# Simple - 2
Solve the following problem step-by-step: 123 - 58

Step-by-step solution:
1. Write your Prompt
2. Write your Prompt
3. Write your Prompt
4. Write your Prompt

Answer: 65
```

위에서 작성한 Simple - 1, Simple - 2 를 few shot 으로 활용하고 아래 문제를 질문으로 하는 프롬프트를 작성하고 결과를 확인해보세요.

```
# Simple 결과 확인
Solve the following problem step-by-step: 345 + 678 - 123

Step-by-step solution:
1. Check the response
2. Check the response
3. Check the response

Answer: 900
```

In [44]:
query_and_print("", """
Let's solve some math problems step-by-step. Follow the thought process for each problem:

# Simple - 1
Solve the following problem step-by-step: 23 + 47

Step-by-step solution:
1. Identify the numbers: We have 23 and 47.
2. Add the ones digit: 3 + 7 = 10. Write down 0 and carry 1.
3. Add the tens digit: 2 + 4 + 1 (carried) = 7.
4. Combine the results: 70 (7 in the tens place, 0 in the ones place).

Answer: 70

# Simple - 2
Solve the following problem step-by-step: 123 - 58

Step-by-step solution:
1. Set up the subtraction: 123 - 58
2. Subtract the ones digit: 3 - 8 is not possible. Borrow 1 from tens, so 13 - 8 = 5.
3. Subtract the tens digit: We borrowed 1, so now it's 1 - 5. Not possible, borrow 1 from hundreds. 11 - 5 = 6.
4. Subtract the hundreds digit: We borrowed 1, so now it's 0 - 0 = 0.

Answer: 65

Now, using the same step-by-step approach, solve this problem:

# Simple 결과 확인
Solve the following problem step-by-step: 345 + 678 - 123

Step-by-step solution:
1. Check the response
2. Check the response
3. Check the response

Answer:
""")

A:
Let's solve the problem step-by-step: 345 + 678 - 123.

Step-by-step solution:

1. **Identify the operations**: We need to first add 345 and 678, and then subtract 123 from the result.

2. **Add 345 and 678**:
   - Align the numbers:
     ```
       345
     + 678
     ------
     ```
   - Add the ones digit: 5 + 8 = 13. Write down 3 and carry over 1.
   - Add the tens digit: 4 + 7 + 1 (carried) = 12. Write down 2 and carry over 1.
   - Add the hundreds digit: 3 + 6 + 1 (carried) = 10. Write down 0 and carry over 1 (which will be placed in the thousands place).
   - Combine the results: 1023 (1 in the thousands place, 0 in the hundreds place, 2 in the tens place, 3 in the ones place).

3. **Now subtract 123 from 1023**:
   - Align the numbers:
     ```
       1023
     -  123
     ------
     ```
   - Subtract the ones digit: 3 - 3 = 0.
   - Subtract the tens digit: 2 - 2 = 0.
   - Subtract the hundreds digit: 0 - 1 is not possible. Borrow 1 from the thousands place, making it 10. S

In [43]:
# 위에서 작성한 Simple - 1, Simple - 2 를 few shot 으로 활용하고 아래 문제를 질문으로 하는 프롬프트를 작성하고 결과를 확인해보세요.
query_and_print("", """
Solve the following problems step-by-step:

# Simple - 1
Solve the following problem step-by-step: 23 + 47

Step-by-step solution:
1. Identify the numbers: We have 23 and 47.
2. Add the ones digits: 3 + 7 = 10. Write down 0 and carry 1.
3. Add the tens digits and the carry: 2 + 4 + 1 (carry) = 7.
4. Combine the results: 70 (7 in the tens place, 0 in the ones place).

Answer: 70

# Simple - 2
Solve the following problem step-by-step: 123 - 58

Step-by-step solution:
1. Set up the subtraction:
   123
   - 58
   ----

2. Subtract the ones place:
   3 - 8 is not possible, so we need to borrow from the tens place.
   13 - 8 = 5
   Write down 5 in the ones place of the result.

3. Subtract the tens place:
   We borrowed 1 from the tens, so we have 1 ten left.
   1 - 5 is not possible, so we need to borrow from the hundreds place.
   11 - 5 = 6
   Write down 6 in the tens place of the result.

4. Subtract the hundreds place:
   We borrowed 1 from the hundreds, so we have 0 hundreds left.
   0 - 0 = 0
   The hundreds place in the result is 0.

Answer: 65

Now, using the same step-by-step approach, solve this problem:

# Simple 결과 확인
Solve the following problem step-by-step: 345 + 678 - 123

Step-by-step solution:
1. 
2. 
3. 

Answer:
""")

A:
Let's solve the problem step-by-step: \( 345 + 678 - 123 \).

### Step-by-step solution:

1. **Identify the numbers**: We have three numbers: 345, 678, and 123.

2. **First, perform the addition**: \( 345 + 678 \).
   - Align the numbers:
     ```
       345
     + 678
     ------
     ```
   - Add the ones digits: \( 5 + 8 = 13 \). Write down 3 and carry over 1.
   - Add the tens digits and the carry: \( 4 + 7 + 1 (carry) = 12 \). Write down 2 and carry over 1.
   - Add the hundreds digits and the carry: \( 3 + 6 + 1 (carry) = 10 \). Write down 0 and carry over 1.
   - Since there are no more digits to add, write down the carry: 1.
   - Combine the results: \( 1023 \).

3. **Now, perform the subtraction**: \( 1023 - 123 \).
   - Align the numbers:
     ```
       1023
     -  123
     ------
     ```
   - Subtract the ones place: \( 3 - 3 = 0 \). Write down 0 in the ones place of the result.
   - Subtract the tens place: \( 2 - 2 = 0 \). Write down 0 in the tens place of the result

아래 문제 (Intermediate-1, Intermediate-2) 를 푸는 과정을 담은 CoT 예시를 작성해보세요.

```
# Intermediate - 1
Solve the following logic puzzle step-by-step:
Three friends, Alice, Bob, and Carol, have different favorite colors: red, blue, and green. We know that:
1. Alice does not like red.
2. Bob does not like blue.
3. Carol likes green.

Determine the favorite color of each friend.

Step-by-step solution:
Write your Prompt

Answer:
- Alice: blue
- Bob: red
- Carol: green
```

```
# Intermediate - 2
Solve the following logic puzzle step-by-step:
Four people (A, B, C, D) are sitting in a row. We know that:
1. A is not next to B.
2. B is next to C.
3. C is not next to D.

Determine the possible seating arrangements.

Step-by-step solution:
Write your Prompt

Answer:
- Possible arrangements: BCAD, CABD
```

In [45]:
query_and_print("", """
# Intermediate - 1
Solve the following logic puzzle step-by-step:
Three friends, Alice, Bob, and Carol, have different favorite colors: red, blue, and green. We know that:
1. Alice does not like red.
2. Bob does not like blue.
3. Carol likes green.

Determine the favorite color of each friend.

Step-by-step solution:
1. Start with the given information:
   - We have three colors: red, blue, and green
   - We have three people: Alice, Bob, and Carol

2. Use the clues to eliminate possibilities:
   - Carol likes green, so we can assign green to Carol
   - Alice does not like red, so Alice must like either blue or green
   - Bob does not like blue, so Bob must like either red or green

3. Deduce the remaining assignments:
   - We know Carol likes green, so Alice can't like green
   - Therefore, Alice must like blue
   - Since Alice likes blue and Carol likes green, Bob must like red

4. Double-check our solution against all clues:
   - Alice likes blue (not red, satisfying clue 1)
   - Bob likes red (not blue, satisfying clue 2)
   - Carol likes green (satisfying clue 3)

Answer:
- Alice: blue
- Bob: red
- Carol: green

# Intermediate - 2
Solve the following logic puzzle step-by-step:
Four people (A, B, C, D) are sitting in a row. We know that:
1. A is not next to B.
2. B is next to C.
3. C is not next to D.

Determine the possible seating arrangements.

Step-by-step solution:
1. Start with the given information:
   - We have four people: A, B, C, D
   - They are sitting in a row (4 positions)

2. Use the clues to determine relative positions:
   - B and C must be next to each other (clue 2)
   - A cannot be next to B (clue 1)
   - D cannot be next to C (clue 3)

3. Deduce possible arrangements:
   - Since B and C are together, they must be in the middle or at one end
   - A and D must be at opposite ends to satisfy all conditions

4. List out the possible arrangements:
   - If B and C are in the middle: A-B-C-D or D-C-B-A
   - But we need to switch B and C in one case to get all possibilities

5. Double-check our solution against all clues:
   - BCAD: Satisfies all clues
   - CABD: Also satisfies all clues

Answer:
- Possible arrangements: BCAD, CABD
""")

A:
Let's break down the second logic puzzle step-by-step to find all possible seating arrangements for A, B, C, and D based on the given clues.

### Step-by-Step Solution:

1. **Start with the given information:**
   - We have four people: A, B, C, D.
   - They are sitting in a row (4 positions).

2. **Use the clues to determine relative positions:**
   - **Clue 1:** A is not next to B.
   - **Clue 2:** B is next to C.
   - **Clue 3:** C is not next to D.

3. **Analyze the implications of the clues:**
   - From **Clue 2**, since B is next to C, we can group them together as (B, C) or (C, B).
   - From **Clue 1**, since A cannot be next to B, A must be positioned away from the (B, C) grouping.
   - From **Clue 3**, since C cannot be next to D, D must also be positioned away from C.

4. **Determine possible arrangements:**
   - The only way to satisfy all clues is to place B and C together and ensure A and D are positioned such that they do not violate the clues.
   - The possible pairs 